In [1]:
import os, shutil
import io
from io import StringIO
import re
import sys
from contextlib import redirect_stdout
from keras import models
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as sp
from scipy.stats import kde
import importlib as im
import cnn_utilities as cn
from sklearn import metrics
import csv

Using TensorFlow backend.


In [3]:
# load trained model and normalization values
my_model = models.load_model("saved_models/train_extant_R0_sampleRate_migrationRate.hdf5")

my_root_model = models.load_model("saved_models/train_extant_root_location.hdf5", compile = False)
my_root_model.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop', 
                      metrics = ['acc', tf.keras.metrics.AUC(), tf.keras.metrics.CategoricalCrossentropy()])

mean_sd = pd.read_csv("saved_models/train_extant_normalization_label_mean_sd.csv", 
                     index_col = 0).to_numpy()

train_means = mean_sd[0,:]
train_sd = mean_sd[1,:]
train_aux_priors_means = train_means[3:,]
train_aux_priors_sd = train_sd[3:,]

num_locs = 5
max_tips = 502


extant_data = pd.read_csv("../model_adequacy/misspec_R0_seir.cblv.csv", 
                   header =None, error_bad_lines = False, index_col = 0).to_numpy()

extant_labels = pd.read_csv("../model_adequacy/misspec_R0_seir_labels.csv",
                    header = None, index_col = 0, error_bad_lines = False).to_numpy()

ERROR! Session/line number was not unique in database. History logging moved to new session 3240


In [4]:
########################################
# comapre APE of CNN to that of phylo ##
########################################

# compute and gather auxilliary prior data
extant_subsample_prop = extant_data[:,(max_tips-1) * 7]
phylocomp_mu = extant_data[:,(max_tips - 3) * 7]
extant_num_tips = cn.get_num_tips(extant_data)

aux_phylocomp = np.vstack((phylocomp_mu, extant_subsample_prop, extant_num_tips,
                          extant_labels[:,8], extant_labels[:,9])).transpose()

norm_aux_phylocomp = cn.normalize(aux_phylocomp, (train_aux_priors_means, train_aux_priors_sd))


# create input tensors
extant_treeLocation_tensor, extant_prior_tensor = cn.create_data_tensors(data = extant_data, 
                                                                                    mu = norm_aux_phylocomp[:,0],
                                                                                    subsample_prop = norm_aux_phylocomp[:,1],
                                                                                    num_tips = norm_aux_phylocomp[:,2],
                                                                                    tmrca = norm_aux_phylocomp[:,3],
                                                                                    mean_bl = norm_aux_phylocomp[:,4],
                                                                                    num_locs = num_locs,
                                                                                    max_tips = max_tips,
                                                                                    cblv_contains_mu_rho = True)

In [5]:
# PREDICT R0, sample rate, migration rate
extant_normalized_preds = my_model.predict([extant_treeLocation_tensor, extant_prior_tensor])

# reversing normalization
cnn_extant_preds = cn.denormalize(extant_normalized_preds, train_means[0:3], train_sd[0:3])
cnn_extant_preds = np.exp(cnn_extant_preds)

# PREDICT root location
extant_root_loc_preds = my_root_model.predict([extant_treeLocation_tensor, extant_prior_tensor])

In [8]:
extant_labels[9,:]


array([0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 7.13095000e+00, 4.76000000e-03, 4.78000000e-03,
       1.00000010e+02, 1.72711036e+01])

In [10]:
cnn_extant_preds[8,:]

array([5.04224774e+00, 4.33535826e-03, 3.79363173e-03])